In [1]:
import pandas as pd
import json

In [2]:
bl_df = pd.read_csv('data/balanced_train_segments.csv', sep=', ')

/tmp/ipykernel_927805/177719126.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  bl_df = pd.read_csv('data/balanced_train_segments.csv', sep=', ')


In [3]:
bl_df.head()

,YTID,start_seconds,end_seconds,positive_labels
0,--PJHxphWEs,30.0,40.0,"""/m/09x0r,/t/dd00088"""
1,--ZhevVpy1s,50.0,60.0,"""/m/012xff"""
2,--aE2O5G5WE,0.0,10.0,"""/m/03fwl,/m/04rlf,/m/09x0r"""
3,--aO5cdqSAg,30.0,40.0,"""/t/dd00003,/t/dd00005"""
4,--aaILOrkII,200.0,210.0,"""/m/032s66,/m/073cg4"""


In [4]:
def make_link(id):
    return f"https://www.youtube.com/watch?v={id}"


In [5]:
bl_df['link'] = bl_df.apply(lambda r: make_link(r['YTID']), axis=1)
bl_df.head()

,YTID,start_seconds,end_seconds,positive_labels,link
0,--PJHxphWEs,30.0,40.0,"""/m/09x0r,/t/dd00088""",https://www.youtube.com/watch?v=--PJHxphWEs
1,--ZhevVpy1s,50.0,60.0,"""/m/012xff""",https://www.youtube.com/watch?v=--ZhevVpy1s
2,--aE2O5G5WE,0.0,10.0,"""/m/03fwl,/m/04rlf,/m/09x0r""",https://www.youtube.com/watch?v=--aE2O5G5WE
3,--aO5cdqSAg,30.0,40.0,"""/t/dd00003,/t/dd00005""",https://www.youtube.com/watch?v=--aO5cdqSAg
4,--aaILOrkII,200.0,210.0,"""/m/032s66,/m/073cg4""",https://www.youtube.com/watch?v=--aaILOrkII


In [6]:
class_labels = pd.read_csv('data/class_labels_indices.csv')
class_labels.head()


,index,mid,display_name
0,0,/m/09x0r,Speech
1,1,/m/05zppz,"Male speech, man speaking"
2,2,/m/02zsn,"Female speech, woman speaking"
3,3,/m/0ytgt,"Child speech, kid speaking"
4,4,/m/01h8n0,Conversation


In [7]:
class_label_dict = {}
for _, row in class_labels.iterrows():
    class_label_dict[row['mid']] = row['display_name']


In [8]:
def get_labels_string(ids):
    ids = ids.replace("\"", '')
    ids = ids.split(',')
    names = [class_label_dict[x] for x in ids]
    return ','.join(names)

In [9]:
bl_df['class_names'] = bl_df.apply(lambda r: get_labels_string(r['positive_labels']), axis=1)
bl_df.head()

,YTID,start_seconds,end_seconds,positive_labels,link,class_names
0,--PJHxphWEs,30.0,40.0,"""/m/09x0r,/t/dd00088""",https://www.youtube.com/watch?v=--PJHxphWEs,"Speech,Gush"
1,--ZhevVpy1s,50.0,60.0,"""/m/012xff""",https://www.youtube.com/watch?v=--ZhevVpy1s,Toothbrush
2,--aE2O5G5WE,0.0,10.0,"""/m/03fwl,/m/04rlf,/m/09x0r""",https://www.youtube.com/watch?v=--aE2O5G5WE,"Goat,Music,Speech"
3,--aO5cdqSAg,30.0,40.0,"""/t/dd00003,/t/dd00005""",https://www.youtube.com/watch?v=--aO5cdqSAg,"Male singing,Child singing"
4,--aaILOrkII,200.0,210.0,"""/m/032s66,/m/073cg4""",https://www.youtube.com/watch?v=--aaILOrkII,"Gunshot, gunfire,Cap gun"


In [10]:
def make_file_path(rownum):
    return f'./data/audios/{rownum}.wav'

# bl_df['file_path'] = 

file_paths = []
for i, _ in bl_df.iterrows():
    file_paths.append(make_file_path(i))

bl_df['file_path'] = file_paths

In [11]:
bl_df.head()

,YTID,start_seconds,end_seconds,positive_labels,link,class_names,file_path
0,--PJHxphWEs,30.0,40.0,"""/m/09x0r,/t/dd00088""",https://www.youtube.com/watch?v=--PJHxphWEs,"Speech,Gush",./data/audios/0.wav
1,--ZhevVpy1s,50.0,60.0,"""/m/012xff""",https://www.youtube.com/watch?v=--ZhevVpy1s,Toothbrush,./data/audios/1.wav
2,--aE2O5G5WE,0.0,10.0,"""/m/03fwl,/m/04rlf,/m/09x0r""",https://www.youtube.com/watch?v=--aE2O5G5WE,"Goat,Music,Speech",./data/audios/2.wav
3,--aO5cdqSAg,30.0,40.0,"""/t/dd00003,/t/dd00005""",https://www.youtube.com/watch?v=--aO5cdqSAg,"Male singing,Child singing",./data/audios/3.wav
4,--aaILOrkII,200.0,210.0,"""/m/032s66,/m/073cg4""",https://www.youtube.com/watch?v=--aaILOrkII,"Gunshot, gunfire,Cap gun",./data/audios/4.wav


In [12]:
from pytube import YouTube, Stream
from threading import Thread
# from concurrent.futures import ThreadPoolExecutor, wait, ALL_COMPLETED
from multiprocessing import Pool

In [13]:
def download_audio_file(link, filename):
    thing: Stream = YouTube(link).streams.filter(only_audio=True, mime_type="audio/mp4").order_by('abr')[0]
    thing.download('./tmp/', filename=filename)

In [14]:
from queue import Queue
from uuid import uuid4
import os
from time import sleep

done = False


def process_file(*args):
    print("In process")
    print(args)

    start, end, fname, tmpname = args

    s_hours = start // 360
    s_minutes = (start % 360) // 60
    s_seconds = start % 60

    command = f"ffmpeg -hide_banner -loglevel error -i ./tmp/{tmpname} -ac 1 -ar 44100 -ss {s_hours:0>2.0f}:{s_minutes:0>2.0f}:{s_seconds:0>2.0f} -t 00:00:10 {fname}"

    while not os.path.exists(f'./tmp/{tmpname}'):
        sleep(0.5)

    os.system(command)
    os.remove(f'./tmp/{tmpname}')


def process_files(q: Queue):
    p = Pool(14)

    while not done or not q.empty():
        stuff = q.get()
        p.apply(process_file, stuff)

    p.close()
    p.join()

process_q = Queue()
download_q = Queue()

for _, row in bl_df.iterrows():
    q_element = (row['link'], row['start_seconds'], row['end_seconds'], row['file_path'])

    download_q.put(q_element)

def download():
    while not download_q.empty():
        try:
            link, start, end, file_path = download_q.get_nowait()
        except Exception as e:
            continue

        tmp_file = f"{uuid4()}.mp4"

        print('Downloading ' + link, flush=True)

        try:
            download_audio_file(link, tmp_file)
        except:
            continue

        print(f'Downloaded {link} as {tmp_file}', flush=True)
        process_q.put((start, end, file_path, tmp_file))


    global done
    done = True

n_download_threads = 20

download_threads = []

for _ in range(n_download_threads):
    t = Thread(target=download)
    t.start()
    download_threads.append(t)


process_files(process_q)

for t in download_threads:
    t.join()




Downloaded https://www.youtube.com/watch?v=--PJHxphWEs as 41aa3cb4-3823-4c60-884b-c82c6aad1df3.mp4
Downloaded https://www.youtube.com/watch?v=-11LhdJgBb8 as c76575a0-12bb-4f9d-9b09-553dff68799b.mp4
In process
Downloaded https://www.youtube.com/watch?v=-1LrH01Ei1w as 07e8c72e-6c54-4f2a-b225-0517b028c406.mp4
(30.0, 40.0, './data/audios/15.wav', 'c76575a0-12bb-4f9d-9b09-553dff68799b.mp4')Downloading https://www.youtube.com/watch?v=-3IYpJfLVJk

In process
Downloaded https://www.youtube.com/watch?v=-0O3e95y4gE as bb56575e-1e07-4118-9b02-9eb6e25cecd6.mp4
(30.0, 40.0, './data/audios/0.wav', '41aa3cb4-3823-4c60-884b-c82c6aad1df3.mp4')Downloading https://www.youtube.com/watch?v=-3hKkjKmIGE

In process
(30.0, 40.0, './data/audios/16.wav', '07e8c72e-6c54-4f2a-b225-0517b028c406.mp4')Downloaded https://www.youtube.com/watch?v=-1iKLvsRBbE as f2c477ac-7fec-49e5-85fa-936f29e33c1b.mp4

In process
Downloaded https://www.youtube.com/watch?v=-0SdAVK79lg as c3605381-812e-4e6f-aa01-c45d9b437a4c.mp4
(100.0,

In [ ]:
for t in download_threads:
    t.join()


KeyboardInterrupt: 